In [ ]:
from dotenv import load_dotenv
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr
from groq import Groq
import traceback

load_dotenv(override=True)

SERPER_API_KEY = os.getenv("SERPER_API_KEY")

def init_clients():
    groq_client = None
    gemini_client = None
    try:
        groq_client = Groq(api_key=os.getenv("GROQ_API_KEY"))
    except Exception as e:
        print("Error initializing Groq:", e)
    try:
        # Try import Gemini client, skip if not available
        from gemini import Gemini
        gemini_client = Gemini(api_key=os.getenv("GOOGLE_API_KEY"))
    except Exception as e:
        print("Error initializing Gemini:", e)
    return groq_client, gemini_client

groq_client, gemini_client = init_clients()

def serper_search(query):
    if not SERPER_API_KEY:
        return "Search API not available."
    url = "https://google.serper.dev/search"
    headers = {"X-API-KEY": SERPER_API_KEY}
    payload = {"q": query}
    try:
        response = requests.post(url, headers=headers, json=payload, timeout=10)
        if response.status_code == 200:
            data = response.json()
            snippet = data.get("organic", [{}])[0].get("snippet", "")
            return snippet or "No relevant information found."
        else:
            return f"Search API error: {response.status_code}"
    except Exception as e:
        return f"Search API exception: {str(e)}"

class AutismAwarenessBot:
    def __init__(self):
        self.name = "Autism Awareness Assistant"
        self.history = []  # list of [user_msg, bot_msg]

        self.autism_knowledge = ""
        try:
            reader = PdfReader("Auticare_chatbot_comprehensivepdf.pdf")
            for page in reader.pages:
                text = page.extract_text()
                if text:
                    self.autism_knowledge += text
        except Exception as e:
            print("Error reading Autism PDF:", e)
            self.autism_knowledge = ""

    def system_prompt(self):
        prompt = (
            f"You are {self.name}, a compassionate and knowledgeable assistant specializing in autism awareness and support.\n\n"
            f"Instructions:\n"
            f"- For the first message, say: \"Hello! Welcome to our autism awareness and support center. I'm here to provide information, resources, and support about autism spectrum disorders. How can I help you today?\"\n"
            f"- PRIMARILY focus on topics related to:\n"
            f"  * Autism spectrum disorders (ASD)\n"
            f"  * Early signs and symptoms of autism\n"
            f"  * Autism diagnosis and assessment\n"
            f"  * Therapies and interventions for autism\n"
            f"  * Supporting individuals with autism\n"
            f"  * Autism awareness and acceptance\n"
            f"  * Resources for families and caregivers\n"
            f"  * Educational strategies for autism\n"
            f"  * Daily living skills and independence\n"
            f"  * Sensory processing and autism\n"
            f"- Use the autism knowledge from the PDF document as your primary source of information\n"
            f"- Be empathetic, understanding, and non-judgmental in all responses\n"
            f"- For current research, statistics, or recent developments in autism, mention you can look up the latest information\n"
            f"- If asked about non-autism topics, gently redirect: \"While I focus on autism awareness and support, I'd be happy to help with any autism-related questions you might have.\"\n"
            f"- Always provide supportive and encouraging responses\n"
            f"- End autism-related advice with: \"For personalized support and resources, consider consulting with autism specialists or local support organizations.\"\n"
            f"- Be respectful of neurodiversity and avoid language that suggests autism needs to be 'cured' or 'fixed'\n\n"
            f"Autism Knowledge Base: {self.autism_knowledge}..."
        )
        return prompt

    def chat(self, message, history):
        if history is None:
            history = []

        # Build messages for LLM call
        messages = [{"role": "system", "content": self.system_prompt()}]
        
        # Add previous conversation history
        for msg in history:
            if msg["role"] == "user":
                messages.append({"role": "user", "content": msg["content"]})
            else:
                messages.append({"role": "assistant", "content": msg["content"]})
        
        # Add current user message
        messages.append({"role": "user", "content": message})

        try:
            if groq_client:
                response = groq_client.chat.completions.create(
                    model="llama3-8b-8192",
                    messages=messages,
                )
                content = response.choices[0].message.content.strip()
            else:
                raise Exception("Groq client not initialized")
        except Exception as e:
            print("Groq error:", e)
            print(traceback.format_exc())
            # Try Gemini fallback if available
            try:
                if gemini_client:
                    response = gemini_client.chat.completions.create(
                        model="gemini-1",
                        messages=messages,
                    )
                    content = response.choices[0].message.content.strip()
                else:
                    raise Exception("Gemini client not initialized")
            except Exception as e2:
                print("Gemini error:", e2)
                print(traceback.format_exc())
                content = (
                    "Sorry, I'm having trouble processing your request right now. "
                    "Please try again and I'll do my best to help you!"
                )

        # If the question seems autism-related and requires current information, try search
        autism_current_keywords = ['latest autism research', 'recent autism studies', 'current autism statistics', 
                                 'new autism therapies', 'autism news', 'recent developments']
        if any(keyword in message.lower() for keyword in autism_current_keywords):
            search_result = serper_search(f"autism {message}")
            if search_result and "No relevant information found" not in search_result and "Search API" not in search_result:
                content += f"\n\nHere's some current information I found about autism:\n{search_result}"

        return content

# Create the autism awareness bot instance
autism_bot = AutismAwarenessBot()

# Create Gradio interface
iface = gr.ChatInterface(
    fn=autism_bot.chat,
    type="messages",
    title="Autism Awareness & Support Assistant",
    description="A compassionate assistant providing information and support about autism spectrum disorders."
)

if __name__ == "__main__":
    iface.launch()